In [23]:
!pip install transformers tokenizers huggingface-hub

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from transformers import pipeline
from nltk import sent_tokenize
import nltk
import torch
from time import time

In [25]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
substitles_paths= sorted(glob('/kaggle/input/narutosubtitle/Subtitles/*.ass'))

In [27]:
scripts = []
episode_num = []
for path in substitles_paths:
    
    with open(path,'r') as file:
        lines = file.readlines()
        lines = lines[27:]
    
        rows = [",".join(line.split(',')[9:]) for line in lines]
        
    rows = [line.replace("\\N",' ') for line in rows]
    script = " ".join(rows)
    
    episode = int(path.split('-')[1].split('.')[0].strip())
    
    scripts.append(script)
    episode_num.append(episode)

In [28]:
df = pd.DataFrame.from_dict({'episode':episode_num,'script':scripts})
df.head()

,episode,script
0,1,"A long time ago, a powerful demon fox appeared..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas..."
2,3,"C'mon!\n Running like a fugitive,\n Being chas..."
3,4,"C'mon!\n Running like a fugitive,\n Being chas..."
4,5,"C'mon!\n Running like a fugitive,\n Being chas..."


In [29]:
device = 0 if torch.cuda.is_available() else 'cpu'

In [30]:
from transformers import pipeline
theme_classifier = pipeline("zero-shot-classification",
                      model= 'facebook/bart-large-mnli',
                      device=device)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [31]:
theme_classifier("I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"],
    multi_label=False
)

{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5227577686309814,
  0.45814040303230286,
  0.014264780096709728,
  0.0026850062422454357,
  0.0021520727314054966]}

# Inference

In [32]:
def get_themes(script):
    script_sentences = sent_tokenize(script)

    # Batch sentences 
    script_batches = []
    sentence_batch_size=20
    for index in range(0,len(script_sentences),sentence_batch_size):
        sent = " ".join(script_sentences[index:index+sentence_batch_size])
        script_batches.append(sent)
    
    # Run model
    candidate_labels = ['friendship','hope', 'sacrifice', 'battle','self development','betrayal','love','dialogue'] 
    emotion_output = theme_classifier(script_batches, 
                       candidate_labels,multi_label=True)

    # Wrangle Output    
    emotions={}
    for output in  emotion_output:
        for label, score in zip(output['labels'],output['scores']):
            if label not in emotions:
                emotions[label]=[]
            emotions[label].append(score)
    
    emotions =  {key:np.mean(np.array(value)) for key,value in emotions.items()}
   
    return emotions

In [ ]:
output_emotions = df['script'].apply(get_themes)

In [ ]:
emotion_df = pd.DataFrame(output_emotions.tolist())
emotion_df.head()

In [ ]:
df[emotion_df.columns] = emotion_df
 
theme_output = df.drop(['episode','script','dialogue'],axis=1).sum().reset_index()
theme_output.columns = ['theme','score']
theme_output

In [ ]:
import seaborn as sns
sns.barplot(data=theme_output, x="theme", y="score")
plt.xticks(rotation=45)
plt.show()